# SALAMI

##Package imports

In [1]:
#Basic imports
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import json

# Imports to support MIR tasks
import mirdata
import jams
import jams.display
import essentia.standard as ess
import librosa
import librosa.display

#Import general signal processing modules
from scipy.fftpack import fft
from math import floor

#Import audio display 
import IPython.display as ipd

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [ ]:
print(f'matplotlib version ---> {mpl.__version__}')
print(f'librosa version ---> {librosa.__version__}')

Let's check out mirdata's supported datasets

In [ ]:
print(f'Is SALAMI available in supported datasets by mirdata? --> {"salami" in mirdata.list_datasets()}')

In [2]:
salami = mirdata.initialize("salami")

In [3]:
salami.data_home = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI'

In [4]:
print(f'Dataset name ---> {salami.name}')
print(f'Bibtex citation ---> {salami.bibtex}')

Dataset name ---> salami
Bibtex citation ---> @inproceedings{smith2011salami,
    title={Design and creation of a large-scale database of structural annotations.},
    author={Smith, Jordan Bennett Louis and Burgoyne, John Ashley and
          Fujinaga, Ichiro and De Roure, David and Downie, J Stephen},
    booktitle={12th International Society for Music Information Retrieval Conference},
    year={2011},
    series = {ISMIR},
}


In [6]:
validation = salami.validate()
print(f'Validation check up ---> {validation}')

100%|██████████| 1359/1359 [00:03<00:00, 409.08it/s]
INFO: Files missing for tracks:
INFO: 602
INFO: 488
INFO: 1477
INFO: 787
INFO: 911
INFO: 1059
INFO: 491
INFO: 742
INFO: 136
INFO: 982
INFO: 1640
INFO: 846
INFO: 1491
INFO: 98
INFO: 1325
INFO: 22
INFO: 979
INFO: 516
INFO: 1232
INFO: 1454
INFO: 743
INFO: 12
INFO: 710
INFO: 427
INFO: 960
INFO: 704
INFO: 915
INFO: 1581
INFO: 479
INFO: 1135
INFO: 1084
INFO: 1536
INFO: 1619
INFO: 644
INFO: 1568
INFO: 1363
INFO: 150
INFO: 757
INFO: 508
INFO: 1371
INFO: 120
INFO: 514
INFO: 691
INFO: 1212
INFO: 599
INFO: 971
INFO: 639
INFO: 242
INFO: 381
INFO: 1335
INFO: 1480
INFO: 27
INFO: 389
INFO: 1204
INFO: 506
INFO: 1275
INFO: 1175
INFO: 176
INFO: 821
INFO: 988
INFO: 924
INFO: 451
INFO: 1605
INFO: 1151
INFO: 1388
INFO: 1479
INFO: 667
INFO: 430
INFO: 618
INFO: 349
INFO: 237
INFO: 1356
INFO: 335
INFO: 1639
INFO: 1330
INFO: 1012
INFO: 1303
INFO: 1429
INFO: 1375
INFO: 935
INFO: 1548
INFO: 620
INFO: 285
INFO: 954
INFO: 1231
INFO: 550
INFO: 936
INFO: 485
INFO:

Validation check up ---> ({'metadata': {}, 'tracks': {'602': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/602.mp3'], '488': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/488.mp3'], '1477': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/1477.mp3'], '787': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/787.mp3'], '911': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/911.mp3'], '1059': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/1059.mp3'], '491': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/491.mp3'], '742': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/742.mp3'], '136': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/136.mp3'], '982': ['/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/982

In [7]:
salami_ids = salami.track_ids  # Return track IDs
salami_data = salami.load_tracks()  # Load Load all tracks in SALAMI

print(type(salami_data[salami_ids[0]])) # Print class
example_track = salami_data[salami_ids[0]]  # Visualize a single track
print(example_track)

<class 'mirdata.datasets.salami.Track'>
Track(
  audio_path="/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/602.mp3",
  sections_annotator1_lowercase_path="...asets/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile1_lowercase.txt",
  sections_annotator1_uppercase_path="...asets/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile1_uppercase.txt",
  sections_annotator2_lowercase_path="...asets/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile2_lowercase.txt",
  sections_annotator2_uppercase_path="...asets/SALAMI/salami-data-public-hierarchy-corrections/annotations/602/parsed/textfile2_uppercase.txt",
  track_id="602",
  annotator_1_id: ,
  annotator_1_time: ,
  annotator_2_id: ,
  annotator_2_time: ,
  artist: ,
  audio: The track's audio

        Returns,
  broad_genre: ,
  duration: ,
  genre: ,
  sections_annotator_1_lowercase: ,
  sections_annotator_1_uppercase: ,
  sec

In [8]:
example_track_audio_path = salami_data[salami_ids[0]].audio_path #track's audio
print(f'Path to example track ---> {example_track_audio_path}')

Path to example track ---> /home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/602.mp3


In [9]:
example_track_audio = salami_data[salami_ids[0]].audio #track's audio
print(f'The actual audio data (array and the sampling rate) of our example track ---> {example_track_audio}')

/opt/conda/envs/master_thesis/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


FileNotFoundError: [Errno 2] No such file or directory: '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/602.mp3'

In [ ]:
#sound reading and displaying
x, fs = example_track_audio
ipd.display(ipd.Audio(data=x, rate=fs))

In [ ]:
# Array preprocessing, X and fft computation
X = fft(x) 
# fft size
N = X.size
# Array length
M = len(x)

# Pre-calculate x-axis range
time_axis = np.arange(N) / float(fs)
freq_axis = np.arange(int(M/2)+1) * float(fs) / N

# Setting figure size
plt.figure(figsize=(20, 10))

# Plotting input sound (x-axis in sec)
plt.subplot(2,1,1)
plt.plot(time_axis, x, 'b')
plt.axis([0, np.max(time_axis), np.min(x), np.max(x)])
plt.ylabel('Amplitude')
plt.xlabel('Time (sec)')
plt.title(str(example_track.audio_path))
plt.autoscale(tight=True)

# Plotting magnitude spectra (x-axis in Hz)
plt.subplot(2,1,2)
plt.plot(freq_axis, np.real(20*np.log10(np.abs(X[:int(M/2)+1]))), 'r', lw=1.5)
plt.xlabel('Frequency domain (Hz)')
plt.ylabel('Amplitude (dB)')
plt.title('Magnitude spectra')
plt.autoscale(tight=True)

plt.show()

The SALAMI dataset provides two levels of reference structural annotations for each track: an upper-level segmentation, where the structural patterns corresponds to sections, and a lower-level segmentation that can reach the phrase level.

In [ ]:
example_ann_lw1 = example_track.sections_annotator1_lowercase_path
example_ann_up1 = example_track.sections_annotator1_uppercase_path
example_ann_lw2 = example_track.sections_annotator2_lowercase_path
example_ann_up2 = example_track.sections_annotator2_uppercase_path

print(f'Path to sections_annotator1_lowercase_path for sample track --> {example_ann_lw1}')
print(f'Path to sections_annotator1_uppercase_path for sample track --> {example_ann_up1}')
print(f'Path to sections_annotator1_lowercase_path for sample track --> {example_ann_lw2}')
print(f'Path to sections_annotator1_uppercase_path for sample track --> {example_ann_up2}')

In [ ]:
with open(example_ann_lw1) as f:
    contents_lw1 = f.readlines()

with open(example_ann_up1) as f:
    contents_up1 = f.readlines()

with open(example_ann_lw2) as f:
    contents_lw2 = f.readlines()

with open(example_ann_up2) as f:
    contents_up2 = f.readlines()
    
print(f'Actual content for "Annotator 1 lowercase for sample track" --> {contents_lw1}')
print(f'Actual content for "Annotator 1 uppercase for sample track "--> {contents_up1}')   
print(f'Actual content for "Annotator 2 lowercase for sample track" --> {contents_lw2}')
print(f'Actual content for "Annotator 2 lowercase for sample track" --> {contents_up2}')

In [ ]:
example_track.to_jams()

In [ ]:
sorted_key_list = example_track.to_jams().annotations[1].data
print(type(sorted_key_list))
print(sorted_key_list)

In [ ]:
df = pd.DataFrame(sorted_key_list)
df

##Metadata descriptions

`metadata.csv`

This is the main metadata file and describes all the pieces in the collection.

The fields are:

	SONG_ID                   Unique identifier for piece of music
	SOURCE                    Either Codaich, IA (Internet Archive), or RWC
	ANNOTATOR1                ID number for first annotator
	ANNOTATOR2                ID number for second annotator
	SONG_DURATION             Duration of the piece, in seconds
	SONG_TITLE                Title
	ARTIST                    Artist
	ANNOTATION_TIME1          Self-reported time to complete annotation for first annotator
	ANNOTATION_TIME2          Self-reported time to complete annotation for second annotator
	TEXTFILE1                 File path for first annotator's file
	TEXTFILE2                 File path for second annotator's file
	CLASS                     Broad genre (classical, jazz, popular, world, Live_Music_Archive, or unknown)
	GENRE                     Narrow genre
	SUBMISSION_DATE1          Date of submission of first annotation
	SUBMISSION_DATE2          Date of submission of second annotation
	XEQS1                     Was the first annotation converted automatically from X/= notation? X indicates yes; 0 indicates no.
	XEQS2                     Was the second annotation converted automatically from X/= notation? X indicates yes; 0 indicates no.

In [ ]:
metadata_csv = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/salami-data-public-hierarchy-corrections/metadata/metadata.csv'
df = pd.read_csv(metadata_csv)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
example_track.get_path